# Evaluating Large Language Models (LLMs)


This notebook demonstrates methods for evaluating LLMs. Focused on the task of summarization and cover accuracy, ROUGE-N, and perplexity.

**Objectives:**
1. Know how to compute ROUGE-N and other metrics.
2. Gain an intuitive understanding of ROUGE-N.
3. Test various models and model sizes on the same data, and compare their results.

## Install and import libraries

In [2]:
!pip install rouge_score -q

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 -q

In [4]:
!pip install datasets -q

In [5]:
!pip install evaluate -q

In [6]:
!pip install nltk -q

In [3]:
!pip install tranformers -q

ERROR: Could not find a version that satisfies the requirement tranformers (from versions: none)
ERROR: No matching distribution found for tranformers


In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Tue_Feb_27_16:28:36_Pacific_Standard_Time_2024
Cuda compilation tools, release 12.4, V12.4.99
Build cuda_12.4.r12.4/compiler.33961263_0


In [13]:
import pandas as pd
import torch
import gc
import os
import transformers as tr
from transformers import AutoTokenizer, T5ForConditionalGeneration, GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
import warnings
warnings.filterwarnings("ignore")

In [10]:
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


## How can we evaluate summarization?

Suppose you are developing a smartphone news app and need to display automatically generated summaries of breaking news articles.  How can you evaluate whether or not the summaries you are generating are good?

## Dataset

We will use a subset of the `cnn_dailymail` dataset from See et al., 2017, downloadable from the [Hugging Face `datasets` hub](https://huggingface.co/datasets/cnn_dailymail).

This dataset provides news article paired with summaries (in the "highlights" column).  Let's load the data and take a look at some examples.

In [11]:
current_directory = os.getcwd()

cache_dir = os.path.join(current_directory, 'cache')

In [14]:
full_dataset = load_dataset(
    "cnn_dailymail", "3.0.0", cache_dir=cache_dir
)  # Note: We specify cache_dir to use pre-cached data.

# Use a small sample of the data during this lab, for speed.
sample_size = 100
sample = (
    full_dataset["train"]
    .filter(lambda r: "CNN" in r["article"][:25])
    .shuffle(seed=42)
    .select(range(sample_size))
)
sample

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 100
})

In [15]:
display(sample.to_pandas())

,article,highlights,id
0,(CNN) -- A magnitude 6.7 earthquake rattled Pa...,Papua New Guinea is on the so-called Ring of F...,8093dba7bc2260c26f18939826909ef27549c758
1,(CNN) -- Pakistan took big steps towards level...,Australia collapse to 88 all out on opening da...,67d626156f971d0bf55e5f2a48e1ed965eb622a6
2,(CNN) -- Federal prosecutors are pushing to fo...,Jared Loughner is refusing the government's re...,0d02fb8f0d406db956b128a5c1cc7bf3f13860a6
3,"Centennial, Colorado (CNN) -- McKayla Hicks sa...",Shooting victim McKayla Hicks went to hearing ...,39aee887c6d34bd311c826142b14037e6f2639ee
4,(CNN) -- Double-amputee sprinter Oscar Pistori...,Oscar Pistorius to become first double-amputee...,cc83ecdf08f0b598c3b97b3e2819c7e0ae7ca4f2
...,...,...,...
95,(CNN) -- Samuel Eto'o netted a superb hat-tric...,Samuel Eto'o scored a hat-trick as Inter Milan...,6c1924f5852b6980a0835877d3f9591a00c70f37
96,Washington (CNN) -- President Barack Obama's r...,Obama raised almost $30 million less than Romn...,0a5691b8fe654b6b2cdace5ab87aff2ee4c23577
97,(CNN) -- Violence swept across Syria on Friday...,NEW: U.N. Secretary-General Ban Ki-moon joins ...,2cc6e4db9758192ac467bbd7424782e4c92206c1
98,(CNN) -- New HIV infections have fallen worldw...,New infections in sub-Saharan Africa 15 percen...,acb2148184f83ecb516ad19a1b0a0e1bc5047237


In [16]:
example_article = sample["article"][0]
example_summary = sample["highlights"][0]
print(f"Article:\n{example_article}\n")
print(f"Summary:\n{example_summary}")


Article:
(CNN) -- A magnitude 6.7 earthquake rattled Papua New Guinea early Friday afternoon, according to the U.S. Geological Survey. The quake was centered about 200 miles north-northeast of Port Moresby and had a depth of 28 miles. No tsunami warning was issued, according to the Tsunami Warning Center. Papua New Guinea is on the so-called Ring of Fire, an arc of fault lines circling the Pacific Basin that is prone to frequent earthquakes and volcanic eruptions.

Summary:
Papua New Guinea is on the so-called Ring of Fire .
It's on an arc of fault lines that is prone to frequent earthquakes .
No tsunami warning was issued .


## Summarization

In [19]:
def batch_generator(data: list, batch_size: int):
    """
    Creates batches of size `batch_size` from a list.
    """
    s = 0
    e = s + batch_size
    while s < len(data):
        yield data[s:e]
        s = e
        e = min(s + batch_size, len(data))


def summarize_with_t5(model_checkpoint: str, articles: list, batch_size: int = 8) -> list:
    """
    Compute summaries using a T5 model.
    This is similar to a `pipeline` for a T5 model but does tokenization manually.

    :param model_checkpoint: Name for a model checkpoint in Hugging Face, such as "t5-small" or "t5-base"
    :param articles: List of strings, where each string represents one article.
    :return: List of strings, where each string represents one article's generated summary
    """
    if torch.cuda.is_available():
        device = "cuda:0"
    else:
        device = "cpu"

    model = T5ForConditionalGeneration.from_pretrained(
        model_checkpoint, cache_dir=cache_dir
    ).to(device)
    tokenizer = AutoTokenizer.from_pretrained(
        model_checkpoint, 
        model_max_length=1024, 
        cache_dir=cache_dir
    )

    def perform_inference(batch: list) -> list:
        inputs = tokenizer(
            batch, max_length=1024, 
            return_tensors="pt", 
            padding=True, 
            truncation=True
        )

        summary_ids = model.generate(
            inputs.input_ids.to(device),
            attention_mask=inputs.attention_mask.to(device),
            num_beams=2,
            min_length=0,
            max_length=40,
        )
        return tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    res = []

    summary_articles = list(map(lambda article: "summarize: " + article, articles))
    for batch in batch_generator(summary_articles, batch_size=batch_size):
        res += perform_inference(batch)

        torch.cuda.empty_cache()
        gc.collect()

    # clean up
    del tokenizer
    del model
    torch.cuda.empty_cache()
    gc.collect()
    return res

In [20]:
t5_small_summaries = summarize_with_t5("t5-small", sample["article"])

In [21]:
reference_summaries = sample["highlights"]

In [22]:
display(
    pd.DataFrame.from_dict(
        {
            "generated": t5_small_summaries,
            "reference": reference_summaries,
        }
    )
)

,generated,reference
0,a magnitude 6.7 earthquake rattles Papua new G...,Papua New Guinea is on the so-called Ring of F...
1,the two-Test cricket series is being played in...,Australia collapse to 88 all out on opening da...
2,federal prosecutors want jared Lee Loughner to...,Jared Loughner is refusing the government's re...
3,"new: ""he tried to kill people,"" a 17-year-old ...",Shooting victim McKayla Hicks went to hearing ...
4,double-amputee sprinter Oscar Pistorius will c...,Oscar Pistorius to become first double-amputee...
...,...,...
95,holders Inter Milan thrash Werder Bremen 4-0 i...,Samuel Eto'o scored a hat-trick as Inter Milan...
96,president's re-election campaign raises $71 mi...,Obama raised almost $30 million less than Romn...
97,"at least 75 people were killed in protests, an...",NEW: U.N. Secretary-General Ban Ki-moon joins ...
98,new infections have fallen by 17 percent in th...,New infections in sub-Saharan Africa 15 percen...


Let's see how our summarization pipeline does!  We'll compute 0/1 accuracy, a classic ML evaluation metric.

In [23]:
accuracy = 0.0
for i in range(len(reference_summaries)):
    generated_summary = t5_small_summaries[i]
    if generated_summary == reference_summaries[i]:
        accuracy += 1.0
accuracy = accuracy / len(reference_summaries)

print(f"Achieved accuracy {accuracy}!")

Achieved accuracy 0.0!


Accuracy zero?!?  We can see that the (very generic) metric of 0/1 accuracy is not useful for summarization. Thinking about this more, small variations in wording may not matter much, and many different summaries may be equally valid. So how can we evaluate summarization?

## ROUGE

 Now that we can generate summaries---and we know 0/1 accuracy is useless here---let's look at how we can compute a meaningful metric designed to evaluate summarization: ROUGE.

Recall-Oriented Understudy for Gisting Evaluation (ROUGE) is a set of evaluation metrics designed for comparing summaries from Lin et al., 2004.  See [Wikipedia](https://en.wikipedia.org/wiki/ROUGE_&#40;metric&#41;) for more info.  Here, we use the Hugging Face Evaluator wrapper to call into the `rouge_score` package.  This package provides 4 scores:

* `rouge1`: ROUGE computed over unigrams (single words or tokens)
* `rouge2`: ROUGE computed over bigrams (pairs of consecutive words or tokens)
* `rougeL`: ROUGE based on the longest common subsequence shared by the summaries being compared
* `rougeLsum`: like `rougeL`, but at "summary level," i.e., ignoring sentence breaks (newlines)

In [27]:
nltk.download('punkt_tab')

rouge_score = evaluate.load("rouge")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aguro\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


You can call `rouge_score` evaluator directly, but below you can see a convenience function to handle the expected input format.


In [28]:
def compute_rouge_score(generated: list, reference: list) -> dict:
    """
    Compute ROUGE scores on a batch of articles.

    This is a convenience function wrapping Hugging Face `rouge_score`,
    which expects sentences to be separated by newlines.

    :param generated: Summaries (list of strings) produced by the model
    :param reference: Ground-truth summaries (list of strings) for comparison
    """
    generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]

    reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]
    
    return rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True,
    )

In [29]:
# ROUGE scores for our batch of articles
compute_rouge_score(t5_small_summaries, reference_summaries)

{'rouge1': 0.30992818372054276,
 'rouge2': 0.10584218670516624,
 'rougeL': 0.22073073087802658,
 'rougeLsum': 0.28214495011248986}

### Understanding ROUGE scores

In [30]:
# Sanity check: What if our predictions match the references exactly?
compute_rouge_score(reference_summaries, reference_summaries)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [31]:
# And what if we fail to predict anything?
compute_rouge_score(
    generated=["" for _ in range(len(reference_summaries))],
    reference=reference_summaries,
)

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}

Stemming predictions and references can help to ignore minor differences.

We will use `rouge_score.compute()` directly for these hand-constructed examples.

In [32]:
rouge_score.compute(
    predictions=["Large language models beat world record"],
    references=["Large language models beating world records"],
    use_stemmer=False,
)

{'rouge1': 0.6666666666666666,
 'rouge2': 0.4000000000000001,
 'rougeL': 0.6666666666666666,
 'rougeLsum': 0.6666666666666666}

In [33]:
rouge_score.compute(
    predictions=["Large language models beat world record"],
    references=["Large language models beating world records"],
    use_stemmer=True,
)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

### ROUGE behaviour in different situations:

In [34]:
# What if we predict exactly 1 word correctly?
rouge_score.compute(
    predictions=["Large language models beat world record"],
    references=["Large"],
    use_stemmer=True,
)

{'rouge1': 0.2857142857142857,
 'rouge2': 0.0,
 'rougeL': 0.2857142857142857,
 'rougeLsum': 0.2857142857142857}

In [35]:
# The ROUGE score is symmetric with respect to predictions and references.
rouge_score.compute(
    predictions=["Large"],
    references=["Large language models beat world record"],
    use_stemmer=True,
)

{'rouge1': 0.2857142857142857,
 'rouge2': 0.0,
 'rougeL': 0.2857142857142857,
 'rougeLsum': 0.2857142857142857}

In [36]:
# What about 2 words?  Note how 'rouge1' and 'rouge2' compare with the case when we predict exactly 2 words correctly.
rouge_score.compute(
    predictions=["Large language"],
    references=["Large language models beat world record"],
    use_stemmer=True,
)


{'rouge1': 0.5, 'rouge2': 0.33333333333333337, 'rougeL': 0.5, 'rougeLsum': 0.5}

In [37]:
# Note how rouge1 differs from the rougeN (N>1) scores when we predict word subsequences correctly.
rouge_score.compute(
    predictions=["Models beat large language world record"],
    references=["Large language models beat world record"],
    use_stemmer=True,
)

{'rouge1': 1.0,
 'rouge2': 0.6,
 'rougeL': 0.6666666666666666,
 'rougeLsum': 0.6666666666666666}

 ## Compare small and large models

  We've been working with the `t5-small` model so far.  Let's compare several models with different architectures in terms of their ROUGE scores and some example generated summaries.


In [38]:
def compute_rouge_per_row(
    generated_summaries: list, reference_summaries: list
) -> pd.DataFrame:
    """
    Generates a dataframe to compare rogue score metrics.
    """
    generated_with_newlines = [
        "\n".join(sent_tokenize(s.strip())) for s in generated_summaries
    ]
    reference_with_newlines = [
        "\n".join(sent_tokenize(s.strip())) for s in reference_summaries
    ]
    scores = rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True,
        use_aggregator=False,
    )
    scores["generated"] = generated_summaries
    scores["reference"] = reference_summaries
    return pd.DataFrame.from_dict(scores)

### T5-small

The [T5](https://huggingface.co/docs/transformers/model_doc/t5) [[paper]](https://arxiv.org/pdf/1910.10683.pdf) family of models are text-to-text transformers that have been trained on a multi-task mixture of unsupervised and supervised tasks. They are well suited for task such as summarization, translation, text classification, question answering, and more.

The t5-small version of the T5 models has 60 million parameters.

In [42]:
# We computed t5_small_summaries above already.
compute_rouge_score(t5_small_summaries, reference_summaries)

{'rouge1': 0.30992818372054276,
 'rouge2': 0.10584218670516624,
 'rougeL': 0.22073073087802658,
 'rougeLsum': 0.28214495011248986}

In [43]:
t5_small_results = compute_rouge_per_row(
    generated_summaries=t5_small_summaries, reference_summaries=reference_summaries
)
display(t5_small_results)

,rouge1,rouge2,rougeL,rougeLsum,generated,reference
0,0.407407,0.230769,0.296296,0.407407,a magnitude 6.7 earthquake rattles Papua new G...,Papua New Guinea is on the so-called Ring of F...
1,0.239130,0.000000,0.195652,0.217391,the two-Test cricket series is being played in...,Australia collapse to 88 all out on opening da...
2,0.454545,0.156250,0.393939,0.454545,federal prosecutors want jared Lee Loughner to...,Jared Loughner is refusing the government's re...
3,0.373333,0.191781,0.266667,0.346667,"new: ""he tried to kill people,"" a 17-year-old ...",Shooting victim McKayla Hicks went to hearing ...
4,0.263158,0.108108,0.184211,0.210526,double-amputee sprinter Oscar Pistorius will c...,Oscar Pistorius to become first double-amputee...
...,...,...,...,...,...,...
95,0.444444,0.285714,0.250000,0.277778,holders Inter Milan thrash Werder Bremen 4-0 i...,Samuel Eto'o scored a hat-trick as Inter Milan...
96,0.320000,0.082192,0.213333,0.320000,president's re-election campaign raises $71 mi...,Obama raised almost $30 million less than Romn...
97,0.155844,0.026667,0.077922,0.155844,"at least 75 people were killed in protests, an...",NEW: U.N. Secretary-General Ban Ki-moon joins ...
98,0.425000,0.102564,0.300000,0.375000,new infections have fallen by 17 percent in th...,New infections in sub-Saharan Africa 15 percen...


### T5-base

The [T5-base](https://huggingface.co/t5-base) model has 220 million parameters.


In [40]:
t5_base_summaries = summarize_with_t5(
    model_checkpoint="t5-base", articles=sample["article"]
)
compute_rouge_score(t5_base_summaries, reference_summaries)


{'rouge1': 0.32733229918837725,
 'rouge2': 0.13303307173288864,
 'rougeL': 0.23660281476457376,
 'rougeLsum': 0.3039145635787135}

In [41]:
t5_base_results = compute_rouge_per_row(
    generated_summaries=t5_base_summaries, reference_summaries=reference_summaries
)
display(t5_base_results)

,rouge1,rouge2,rougeL,rougeLsum,generated,reference
0,0.452830,0.352941,0.264151,0.452830,the quake was centered about 200 miles north-n...,Papua New Guinea is on the so-called Ring of F...
1,0.341463,0.125000,0.195122,0.292683,Pakistan reach 148-3 on opening day of two-Tes...,Australia collapse to 88 all out on opening da...
2,0.347826,0.179104,0.260870,0.347826,federal prosecutors want a handwriting sample ...,Jared Loughner is refusing the government's re...
3,0.083333,0.000000,0.083333,0.083333,"""he's just this freak lookin' dude with some o...",Shooting victim McKayla Hicks went to hearing ...
4,0.378378,0.166667,0.216216,0.324324,double-amputee sprinter Oscar Pistorius will c...,Oscar Pistorius to become first double-amputee...
...,...,...,...,...,...,...
95,0.472222,0.342857,0.416667,0.444444,Samuel Eto'o nets hat-trick as Inter Milan thr...,Samuel Eto'o scored a hat-trick as Inter Milan...
96,0.278481,0.077922,0.177215,0.253165,the president's re-election campaign released ...,Obama raised almost $30 million less than Romn...
97,0.133333,0.000000,0.106667,0.133333,new: a doctor says six people were killed and ...,NEW: U.N. Secretary-General Ban Ki-moon joins ...
98,0.368421,0.135135,0.236842,0.368421,new infections have fallen by 17 percent over ...,New infections in sub-Saharan Africa 15 percen...


### GPT-2

The [GPT-2](https://huggingface.co/gpt2) model is a generative text model that was trained in a self-supervised fashion. Its strengths are in using a 'completing the sentence' for a given prompt.  It has 124 million parameters.


In [46]:
def summarize_with_gpt2(
    model_checkpoint: str, articles: list, batch_size: int = 8
) -> list:
    """
    Convenience function for summarization with GPT2 to handle these complications:
    - Append "TL;DR" to the end of the input to get GPT2 to generate a summary.
    https://huggingface.co/course/chapter7/5?fw=pt
    - Truncate input to handle long articles.
    - GPT2 uses a max token length of 1024.  We use a shorter 512 limit here.

    :param model_checkpoint: reference to checkpointed model
    :param articles: list of strings
    :return: generated summaries, with the input and "TL;DR" removed
    """
    if torch.cuda.is_available():
        device = "cuda:0"
    else:
        device = "cpu"

    tokenizer = GPT2Tokenizer.from_pretrained(
        model_checkpoint, padding_side="left", cache_dir=cache_dir
    )
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    model = GPT2LMHeadModel.from_pretrained(
        model_checkpoint,
        pad_token_id=tokenizer.eos_token_id,
        cache_dir=cache_dir,
    ).to(device)

    def perform_inference(batch: list) -> list:
        tmp_inputs = tokenizer(
            batch, max_length=500, return_tensors="pt", padding=True, truncation=True
        )
        tmp_inputs_decoded = tokenizer.batch_decode(
            tmp_inputs.input_ids, skip_special_tokens=True
        )
        inputs = tokenizer(
            [article + " TL;DR:" for article in tmp_inputs_decoded],
            max_length=512,
            return_tensors="pt",
            padding=True,
            truncation=True,
        )
        summary_ids = model.generate(
            inputs.input_ids.to(device),
            attention_mask=inputs.attention_mask.to(device),
            num_beams=2,
            min_length=0,
            max_length=512 + 32,
        )
        return tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    decoded_summaries = []
    for batch in batch_generator(articles, batch_size=batch_size):
        decoded_summaries += perform_inference(batch)

        # batch clean up
        torch.cuda.empty_cache()
        gc.collect()

    # post-process decoded summaries
    summaries = [
        summary[summary.find("TL;DR:") + len("TL;DR: ") :]
        for summary in decoded_summaries
    ]

    # cleanup
    del tokenizer
    del model
    torch.cuda.empty_cache()
    gc.collect()

    return summaries

In [47]:
gpt2_summaries = summarize_with_gpt2(
    model_checkpoint="gpt2", articles=sample["article"]
)
compute_rouge_score(gpt2_summaries, reference_summaries)


{'rouge1': 0.19606083904686428,
 'rouge2': 0.04639299343246664,
 'rougeL': 0.14395802138003508,
 'rougeLsum': 0.1792659399398965}

In [48]:
gpt2_results = compute_rouge_per_row(
    generated_summaries=gpt2_summaries, reference_summaries=reference_summaries
)
display(gpt2_results)

,rouge1,rouge2,rougeL,rougeLsum,generated,reference
0,0.295082,0.067797,0.196721,0.229508,A magnitude 6.7 earthquake rattled Papua New G...,Papua New Guinea is on the so-called Ring of F...
1,0.108696,0.022222,0.108696,0.086957,"Australia lost the toss. ""I think it was a big...",Australia collapse to 88 all out on opening da...
2,0.513514,0.138889,0.324324,0.432432,The government wants to force Jared Lee Loughn...,Jared Loughner is refusing the government's re...
3,0.144578,0.024691,0.120482,0.120482,I'm not going to be able to do anything about ...,Shooting victim McKayla Hicks went to hearing ...
4,0.244444,0.045455,0.200000,0.244444,"""The CAS has determined that the athlete's per...",Oscar Pistorius to become first double-amputee...
...,...,...,...,...,...,...
95,0.078431,0.000000,0.039216,0.078431,Lyon win 3-1.\n\nRead More,Samuel Eto'o scored a hat-trick as Inter Milan...
96,0.071429,0.000000,0.047619,0.071429,The Obamas won't be going on vacation to Blue ...,Obama raised almost $30 million less than Romn...
97,0.144578,0.024691,0.120482,0.144578,"""The government is not going to allow us to go...",NEW: U.N. Secretary-General Ban Ki-moon joins ...
98,0.195122,0.000000,0.121951,0.195122,"HIV/AIDS is not a disease that can be cured, t...",New infections in sub-Saharan Africa 15 percen...


### Comparing all models

We use a couple of helper functions to compare the above models, first by their evaluation metrics (quantitative) and second by their generated summaries (qualitative).


In [49]:
def compare_models(models_results: dict) -> pd.DataFrame:
    """
    :param models_results: dict of "model name" string mapped to pd.DataFrame of results computed by `compute_rouge_per_row`
    :return: pd.DataFrame with 1 row per model, with columns: model, rouge1, rouge2, rougeL, rougeLsum
    where metrics are averages over input results for each model
    """
    agg_results = []
    for r in models_results:
        model_results = models_results[r].drop(
            labels=["generated", "reference"], axis=1
        )
        agg_metrics = [r]
        agg_metrics[1:] = model_results.mean(axis=0)
        agg_results.append(agg_metrics)
    return pd.DataFrame(
        agg_results, columns=["model", "rouge1", "rouge2", "rougeL", "rougeLsum"]
    )


In [50]:
display(
    compare_models(
        {
            "t5-small": t5_small_results,
            "t5-base": t5_base_results,
            "gpt2": gpt2_results,
        }
    )
)

,model,rouge1,rouge2,rougeL,rougeLsum
0,t5-small,0.309642,0.106133,0.221509,0.282184
1,t5-base,0.327103,0.133288,0.237264,0.303476
2,gpt2,0.195796,0.046363,0.144114,0.179105


In [51]:
def compare_models_summaries(models_summaries: dict) -> pd.DataFrame:
    """
    Aggregates results from `models_summaries` and returns a dataframe.
    """
    comparison_df = None
    for model_name in models_summaries:
        summaries_df = models_summaries[model_name]
        if comparison_df is None:
            comparison_df = summaries_df[["generated"]].rename(
                {"generated": model_name}, axis=1
            )
        else:
            comparison_df = comparison_df.join(
                summaries_df[["generated"]].rename({"generated": model_name}, axis=1)
            )
    return comparison_df


In [52]:
# In the output table below, scroll to the right to see all models.
display(
    compare_models_summaries(
        {
            "t5_small": t5_small_results,
            "t5_base": t5_base_results,
            "gpt2": gpt2_results,
        }
    )
)

,t5_small,t5_base,gpt2
0,a magnitude 6.7 earthquake rattles Papua new G...,the quake was centered about 200 miles north-n...,A magnitude 6.7 earthquake rattled Papua New G...
1,the two-Test cricket series is being played in...,Pakistan reach 148-3 on opening day of two-Tes...,"Australia lost the toss. ""I think it was a big..."
2,federal prosecutors want jared Lee Loughner to...,federal prosecutors want a handwriting sample ...,The government wants to force Jared Lee Loughn...
3,"new: ""he tried to kill people,"" a 17-year-old ...","""he's just this freak lookin' dude with some o...",I'm not going to be able to do anything about ...
4,double-amputee sprinter Oscar Pistorius will c...,double-amputee sprinter Oscar Pistorius will c...,"""The CAS has determined that the athlete's per..."
...,...,...,...
95,holders Inter Milan thrash Werder Bremen 4-0 i...,Samuel Eto'o nets hat-trick as Inter Milan thr...,Lyon win 3-1.\n\nRead More
96,president's re-election campaign raises $71 mi...,the president's re-election campaign released ...,The Obamas won't be going on vacation to Blue ...
97,"at least 75 people were killed in protests, an...",new: a doctor says six people were killed and ...,"""The government is not going to allow us to go..."
98,new infections have fallen by 17 percent in th...,new infections have fallen by 17 percent over ...,"HIV/AIDS is not a disease that can be cured, t..."
